<a href="https://colab.research.google.com/github/geovalexis/TFG/blob/main/Methods.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Current resources information



In [34]:
!cat /proc/cpuinfo
import threading
threading.active_count()

processor	: 0
vendor_id	: GenuineIntel
cpu family	: 6
model		: 63
model name	: Intel(R) Xeon(R) CPU @ 2.30GHz
stepping	: 0
microcode	: 0x1
cpu MHz		: 2299.998
cache size	: 46080 KB
physical id	: 0
siblings	: 2
core id		: 0
cpu cores	: 1
apicid		: 0
initial apicid	: 0
fpu		: yes
fpu_exception	: yes
cpuid level	: 13
wp		: yes
flags		: fpu vme de pse tsc msr pae mce cx8 apic sep mtrr pge mca cmov pat pse36 clflush mmx fxsr sse sse2 ss ht syscall nx pdpe1gb rdtscp lm constant_tsc rep_good nopl xtopology nonstop_tsc cpuid tsc_known_freq pni pclmulqdq ssse3 fma cx16 pcid sse4_1 sse4_2 x2apic movbe popcnt aes xsave avx f16c rdrand hypervisor lahf_lm abm invpcid_single ssbd ibrs ibpb stibp fsgsbase tsc_adjust bmi1 avx2 smep bmi2 erms invpcid xsaveopt arat md_clear arch_capabilities
bugs		: cpu_meltdown spectre_v1 spectre_v2 spec_store_bypass l1tf mds swapgs
bogomips	: 4599.99
clflush size	: 64
cache_alignment	: 64
address sizes	: 46 bits physical, 48 bits virtual
power management:

processor	:

5

# Data retrievement


## Reference Proteome Dataset

In [ ]:
# Quest for Orthologs (QfO) dataset
!ls drive/MyDrive/TFG/QFO_2018/

# DEPRECATED
# QfO species and statistics
# We got this information from README file of the QfO release 
# It was formatted into a tabulated file by the following command: 
# cat QfO_statistics.txt | tr -s ' ' | cut -f1-6 -d" " | tr ' ' '\t' > QfO_statistics.tsv
!ls drive/MyDrive/TFG/QFO_2018/QfO_statistics.tsv


QfO_statistics.tsv  UP000005640_9606.fasta     UP000005640_9606.idmapping
README		    UP000005640_9606.gene2acc  UP000005640_9606.xml
drive/MyDrive/TFG/QFO_2018/QfO_statistics.tsv
     |████████████████████████████████| 2.3MB 4.7MB/s 
A0A024R161
A0A024R1R8
A0A075B6F4
A0A075B6H5
A0A075B6H7
A0A075B6H8
A0A075B6H9
A0A075B6I0
A0A075B6I1
A0A075B6I3


In [127]:
# Get QfO species (taxIDs) and all their corresponding uniprotIDs available in the QfO database

#!wget ftp://ftp.ebi.ac.uk/pub/databases/reference_proteomes/previous_releases/qfo_release-2018_04/QfO_release_2018_04.tar.gz 
#!tar -xvzf QfO_release_2018_04.tar.gz --one-top-level

import os
import pandas as pd
qfoDir='./QfO_release_2018_04/'
QfO_proteome = {}
for subdir, dirs, files in os.walk(qfoDir):
    for file in files:
        if file.endswith(".gene2acc"):
            taxID = file.split("_")[1].split(".")[0]
            QfO_proteome[taxID] = pd.read_csv(os.path.join(subdir, file), sep="\t", names=["GeneSymbol", "UniprotKB", "CanonicalGeneSymbol"], usecols=["UniprotKB"])["UniprotKB"]
QfO_proteome = pd.DataFrame(QfO_proteome)
QfO_proteome.head()
QfO_proteome.to_csv("drive/MyDrive/TFG/QFO_2018/QfO_uniprotKBs.tsv", sep="\t", header=True, index=False)
!head drive/MyDrive/TFG/QFO_2018/QfO_uniprotKBs.tsv

243232	188937	273057	69014	64091	374847	436308	243273	251221	1111708	189518	122586	224911	243230	243231	190304	224324	208964	289376	324602	243090	85962	100226	243274	83332	224308	83333	226186	515635	272561	6412	39947	9031	6239	184922	9595	45351	13616	237561	367110	4577	81824	214684	418459	6945	5888	237631	9913	9615	36329	5664	7165	321614	5722	284591	7070	9606	44689	7918	284812	8364	35128	10116	684364	7955	7739	8090	9598	7227	7719	3055	10090	3702	665079	3218	164328	559292	330879
Q60328	Q8TUR2	Q97TX9	P77933	O51955	B1L7D4	A9A4Z2	P47377	Q7NPQ4	P77971	Q8FA34	Q9K1R8	Q89YF1	Q9RYE8	Q74GG6	Q8RHA7	O67904	Q9I7C5	B5YGS5	A9WAN1	Q7UZ97	O24866	O50499	Q9WYH1	P71591	P05648	P23843	Q8ABV9	B8DYG0	O84004	T1ECX9	Q8RZL9	P40666	P34257	A8B2Y5	P30375	A7S0L9	Q28462	A0A1D8PCG7	P22702	P19950	A9V4W6	Q5KQA0	E3JPW3	B7P404	A0BAJ9	Q0H8Y6	Q2M2T3	O97945	Q25823	Q01782	Q7QEI4	A7UG00	A2GII8	Q36258	Q94R76	Q6P435	Q23890	W5LV44	Q9US57	P84383	B8BT19	P01681	F4NY12	P0C8Y2	O47426	Q01798	P30686	Q8T8R6	F7A355	P35006	P01753	Q9SP32	A7

In [126]:
# Get species taxIDs from QfO (it comes with many strain or subspecie taxIDs)

!pip install --upgrade ete3
from ete3 import NCBITaxa
import pandas as pd

# The first line (header) of the tsv file corresponds to the taxIDs of the reference species
QfO_reference_taxa = pd.read_csv("drive/MyDrive/TFG/QFO_2018/QfO_uniprotKBs.tsv", sep="\t", nrows=0).columns.astype("int32")
QfO_reference_species = [(QfO_reference_taxa[i], *getSpecie(QfO_reference_taxa[i])) for i in range(QfO_reference_taxa.size)]
QfO_reference_species = pd.DataFrame(QfO_reference_species, columns=["QfO_taxID", "specie_taxID", "species_name"])
QfO_reference_species.to_csv("drive/MyDrive/TFG/QFO_2018/subspecieID2specieID.tsv", sep="\t", header=True, index=False)
!head drive/MyDrive/TFG/QFO_2018/subspecieID2specieID.tsv

# Snippet got from mapQfO2MtP.py (Gabaldonlab/qfo-2020 repo) but slightly modified to also output specie's name
def getSpecie(taxID: int):
    """This function searches the corresponding specie taxID for a given strain or subspecie taxID (MUST BE AN INTEGER) in the NCBI database. 
    Args:
        taxID (int): taxID of the taxon of interest.
    Returns:
        specieID (int): taxID of the corresponding specie.
    """
    
    ncbi = NCBITaxa()
    #ncbi.update_taxonomy_database() 
    
    specieID = taxID # If the taxID is already a specie it will return the same taxID
    if ncbi.get_rank([taxID])[taxID] != 'species':
        lineage = ncbi.get_lineage(taxID)
        for j in reversed(lineage): #Reverse because it's faster when it's a strain or subspecie
            if ncbi.get_rank([j])[j] == 'species':
                specieID = j
    species_name = ncbi.get_taxid_translator([specieID])[specieID]
    return specieID, species_name


Requirement already up-to-date: ete3 in /usr/local/lib/python3.6/dist-packages (3.1.2)
QfO_taxID	specie_taxID	species_name
243232	2190	Methanocaldococcus jannaschii
188937	2214	Methanosarcina acetivorans
273057	2287	Saccharolobus solfataricus
69014	311400	Thermococcus kodakarensis
64091	2242	Halobacterium salinarum
374847	498846	Candidatus Korarchaeum cryptofilum
436308	338192	Nitrosopumilus maritimus
243273	2097	Mycoplasma genitalium
251221	33072	Gloeobacter violaceus


In [64]:
# Human referece proteome dataset - CANONICAL PROTEOME
# These are only the canonical proteins (in the xml are only the canonical sequences).
# We are not adding isoforms or variants proteins so far.
# This is the reason why we do not take the uniprotIDs directly from the previous file.

!pip install biopython
from Bio import SeqIO
import pandas as pd
records = list(SeqIO.parse("drive/MyDrive/TFG/QFO_2018/UP000005640_9606.xml", "seqxml"))
records_ids = [record.id for record in records]
df_records_ids = pd.DataFrame(records_ids)
df_records_ids.to_csv("drive/MyDrive/TFG/QFO_2018/human_reference_proteome.tsv", sep="\t", index=False, header=False)
!head drive/MyDrive/TFG/QFO_2018/human_reference_proteome.tsv

A0A024R161
A0A024R1R8
A0A075B6F4
A0A075B6H5
A0A075B6H7
A0A075B6H8
A0A075B6H9
A0A075B6I0
A0A075B6I1
A0A075B6I3


## Orthologs retrievement

### From MetaPhors

In [5]:
!ls drive/MyDrive/TFG/QfO_input.tsv #Orthologs input for QfO

drive/MyDrive/TFG/QfO_input.tsv


## Matrix


### MetaPhOrs orthologs between homo sapiens and the other 78 reference species from the Quest for Orthologs


In [ ]:
%%time
!pip install aiohttp
!pip install asyncio

import pandas as pd
import numpy as np
import aiohttp
import asyncio

# TO SOLVE "This event loop is already running in python" error
!pip install nest_asyncio
import nest_asyncio
nest_asyncio.apply()

mtp_orthologs = pd.read_csv("drive/MyDrive/TFG/QfO_input.tsv", sep="\t", names=["protein1", "protein2"])
qfo_ref_human_proteome = pd.read_csv("drive/MyDrive/TFG/QFO_2018/human_reference_proteome.tsv", sep="\t", names=["protein1"])
print("Total size of the orthologs file from MetaPhOrs:", mtp_orthologs.size)
print("Number of reference proteins (from QfO):", qfo_ref_human_proteome.size)
ref_protein_mtp = pd.merge(mtp_orthologs, qfo_ref_human_proteome, how="inner", on="protein1")
print("Size of the resulting inner join between the two datasets:", ref_protein_mtp.size)
print("Actual number of reference Homo Sapiens proteins within MetaPhOrs:", ref_protein_mtp["protein1"].unique().size)
print("First 10 rows:\n", ref_protein_mtp.head(10))
#ref_protein_mtp.drop_duplicates() # There just around 200 repeated rows but I think that they correspond to the orthologs within the same specie (Homo sapiens in this case) -> diagonal

async def map_uniprotIDs2taxIDs_EBIRequest(uniprotID, session):
    # Documentation in https://www.ebi.ac.uk/proteins/api/doc/
    requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprotID}"
    response = await session.get(requestURL, headers={ "Accept" : "application/json"})
    if response.ok: # status_code == 200
        return (await response.json())["organism"]["taxonomy"]
    else:
        print(f"Protein {uniprotID} couldn't be found")
        return np.nan

async def map_uniprotIDs2taxIDs_EBIRequest_multiprocessing(uniprotIDs):
  async with aiohttp.ClientSession() as session:
    return await asyncio.gather(*[map_uniprotIDs2taxIDs_EBIRequest(uniprotID, session) for uniprotID in uniprotIDs])

loop = asyncio.get_event_loop()
ref_protein_mtp["orthologs_taxID"] = pd.Series(loop.run_until_complete(map_uniprotIDs2taxIDs_EBIRequest_multiprocessing(ref_protein_mtp["protein2"].to_list())), dtype=pd.Int64Dtype())
ref_protein_mtp.to_csv("drive/MyDrive/TFG/human_orthologs_MtP-QfO2018.tsv", sep="\t", index=False, header=["human_protein", "ortholog", "orthologs_taxID"])
#pd.DataFrame(ref_protein_mtp["protein1"].unique()).to_csv("drive/MyDrive/TFG/QFO_2018/human_reference_proteome_MtP.tsv", sep="\t", index=False, header=False)
!head drive/MyDrive/TFG/human_orthologs_MtP-QfO2018.tsv



Total size of the orthologs file from MetaPhOrs: 7898268
Number of reference proteins (from QfO): 20996
Size of the resulting inner join between the two datasets: 139718
Actual number of reference Homo Sapiens proteins within MetaPhOrs: 5254
First 10 rows:
   protein1    protein2
0   Q8ND71      Q7ZAM9
1   Q8ND71      Q6CDT3
2   Q8ND71      A2ESR8
3   Q8ND71      A2E1H0
4   Q8ND71      A2FTJ3
5   Q8ND71      A2FTJ2
6   Q8ND71      G3S4T9
7   Q8ND71  A0A2I3SJI4
8   Q8ND71      H2R4V7
9   P15170      Q8F6D0
Protein F7B6L4 couldn't be found
Protein F6W3J1 couldn't be found
Protein A0A0G2KT54 couldn't be found
Protein F7D6T9 couldn't be found
Protein H2RE68 couldn't be found
Protein F1QV09 couldn't be found
Protein L7N0F6 couldn't be found
Protein R4GMA0 couldn't be found
Protein F6XNB2 couldn't be found
Protein F7CX40 couldn't be found
Protein F6Y7V8 couldn't be found
Protein F7C8Q0 couldn't be found
Protein K7E3L4 couldn't be found
Protein F6T461 couldn't be found
Protein F7CVN2 couldn't

In [40]:
%%time
import numpy as np
import requests

def map_uniprotIDs2taxIDs_BruteForce(uniprotIDs):
  qfo_uniprotIDs = pd.read_csv("drive/MyDrive/TFG/QFO_2018/QfO_uniprotKBs.tsv", sep="\t", header=0, dtype="string")
  qfo_uniprotIDs.fillna("-", inplace=True) #To solve "boolean value of NA is ambiguous pandas" error
  taxIDs = []
  for uniprotID in uniprotIDs:
    found = False
    for taxID in qfo_uniprotIDs:
        if uniprotID in qfo_uniprotIDs[taxID].unique():
            taxIDs.append(taxID)
            found = True
            break
    if not found:
        taxIDs.append(np.nan)
  return taxIDs

def map_uniprotIDs2taxIDs_UniprotRequest(uniprotIDs):
    # Documentation in https://www.uniprot.org/help/api_queries
    taxIDs = []
    endpoint = "https://www.uniprot.org/uniprot/"
    for uniprotID in uniprotIDs:
        params = {
            'query': uniprotID,
            'columns': 'organism-id', #Other information can be added (just add the field in comma-separated). Ex. 'id,organism-id,genes'
            'format': 'tab'
        }
        response = requests.get(endpoint, params=params)
        if response.status_code == 200:
            taxIDs.append(response.text.splitlines()[1])
        else:
            print(f"The tax for {uniprotID} couldn't be found")
            taxIDs.append(np.nan)
    return taxIDs

def map_uniprotIDs2taxIDs_EBIRequest(uniprotIDs):
    # Documentation in https://www.ebi.ac.uk/proteins/api/doc/
    taxIDs = []
    for uniprotID in uniprotIDs:
        requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprotID}"
        response = requests.get(requestURL, headers={ "Accept" : "application/json"})
        if response.ok: # status_code == 200
            taxIDs.append(response.json()["organism"]["taxonomy"])
        else:
            print(f"The tax for {uniprotID} couldn't be found")
            taxIDs.append(np.nan)
    return taxIDs

print(map_uniprotIDs2taxIDs_EBIRequest(["Q7ZAM9", "Q8F6D0", "Q8F6A3", "Q8F692", "Q8F624", "Q7L211", "Q8CXT3"]))


[189518, 189518, 189518, 189518, 189518, 9606, 189518]
CPU times: user 93.4 ms, sys: 14.9 ms, total: 108 ms
Wall time: 4 s


In [41]:
# MULTIPROCESSING VERSION OF map_uniprotIDs2taxIDs_EBIRequest

#!pip install aiohttp
#!pip install asyncio

# TO SOLVE "This event loop is already running in python" error
#!pip install nest_asyncio
#import nest_asyncio
#nest_asyncio.apply()
%%time


import aiohttp
import asyncio

async def map_uniprotIDs2taxIDs_EBIRequest(uniprotID, session):
    # Documentation in https://www.ebi.ac.uk/proteins/api/doc/
    requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprotID}"
    response = await session.get(requestURL, headers={ "Accept" : "application/json"})
    if response.ok: # status_code == 200
        return (await response.json())["organism"]["taxonomy"]
    else:
        return np.nan

async def map_uniprotIDs2taxIDs_EBIRequest_multiprocessing(uniprotIDs):
  async with aiohttp.ClientSession() as session:
    return await asyncio.gather(*[map_uniprotIDs2taxIDs_EBIRequest(uniprotID, session) for uniprotID in uniprotIDs])

loop = asyncio.get_event_loop()
taxIDs = loop.run_until_complete(map_uniprotIDs2taxIDs_EBIRequest_multiprocessing(["Q7ZAM9", "Q8F6D0", "Q8F6A3", "Q8F692", "Q8F624", "Q7L211", "Q8CXT3"]))
print(taxIDs)

[189518, 189518, 189518, 189518, 189518, 9606, 189518]
CPU times: user 22 ms, sys: 2.28 ms, total: 24.3 ms
Wall time: 607 ms


In [32]:
%%time
import numpy as np
import requests

def map_uniprotIDs2taxIDs_BruteForce(uniprotIDs):
  qfo_uniprotIDs = pd.read_csv("drive/MyDrive/TFG/QFO_2018/QfO_uniprotKBs.tsv", sep="\t", header=0, dtype="string")
  qfo_uniprotIDs.fillna("-", inplace=True) #To solve "boolean value of NA is ambiguous pandas" error
  taxIDs = []
  for uniprotID in uniprotIDs:
    found = False
    for taxID in qfo_uniprotIDs:
        if uniprotID in qfo_uniprotIDs[taxID].unique():
            taxIDs.append(taxID)
            found = True
            break
    if not found:
        taxIDs.append(np.nan)
  return taxIDs

def map_uniprotIDs2taxIDs_UniprotRequest(uniprotIDs):
    # Documentation in https://www.uniprot.org/help/api_queries
    taxIDs = []
    endpoint = "https://www.uniprot.org/uniprot/"
    for uniprotID in uniprotIDs:
        params = {
            'query': uniprotID,
            'columns': 'organism-id', #Other information can be added (just add the field in comma-separated). Ex. 'id,organism-id,genes'
            'format': 'tab'
        }
        response = requests.get(endpoint, params=params)
        if response.status_code == 200:
            taxIDs.append(response.text.splitlines()[1])
        else:
            print(f"The tax for {uniprotID} couldn't be found")
            taxIDs.append(np.nan)
    return taxIDs

def map_uniprotIDs2taxIDs_EBIRequest(uniprotIDs):
    # Documentation in https://www.ebi.ac.uk/proteins/api/doc/
    taxIDs = []
    endpoint = "https://www.uniprot.org/uniprot/"
    for uniprotID in uniprotIDs:
        requestURL = f"https://www.ebi.ac.uk/proteins/api/proteins/{uniprotID}"
        response = requests.get(requestURL, headers={ "Accept" : "application/json"})
        if response.ok: # status_code == 200
            taxIDs.append(response.json()["organism"]["taxonomy"])
        else:
            print(f"The tax for {uniprotID} couldn't be found")
            taxIDs.append(np.nan)
    return taxIDs

print(map_uniprotIDs2taxIDs_EBIRequest(["Q7ZAM9", "Q8F6D0", "Q8F6A3", "Q8F692", "Q8F624", "Q7L211", "Q8CXT3"]))


['189518', '189518', '189518', '189518', '189518', '9606', '189518']
CPU times: user 77.3 ms, sys: 4.61 ms, total: 81.9 ms
Wall time: 2.02 s


#Phylogenetic Profiling (PP)
